In [2]:
import os, json, requests
from pathlib import Path
import mlflow

# --- 1) Subir al root del repositorio ---
def goto_repo_root(max_hops=7):
    here = Path.cwd()
    for _ in range(max_hops):
        if (here / ".git").exists():
            os.chdir(here)
            return here
        here = here.parent
    return Path.cwd()

repo_root = goto_repo_root()
print("📂 Repo root:", repo_root)

# --- 2) Cargar TOKEN desde .env ---
try:
    from dotenv import load_dotenv, find_dotenv
    envp = find_dotenv(filename=".env", usecwd=True)
    if envp:
        load_dotenv(dotenv_path=envp, override=True)
    else:
        print("⚠️ .env no encontrado (solo estamos usando $env del sistema)")
except Exception as e:
    print("dotenv optional:", e)

# --- 3) Cargar HOST desde databricks_host.txt (versionado) ---
host_file = repo_root / "databricks_host.txt"
if host_file.exists():
    h = host_file.read_text(encoding="utf-8").strip()
    # limpiar BOM si existiera
    h2 = h.lstrip("\ufeff").strip()
    os.environ["DATABRICKS_HOST"] = h2
else:
    print("⚠️ No existe databricks_host.txt")

# --- 4) Validar credenciales ---
host  = os.getenv("DATABRICKS_HOST")
token = os.getenv("DATABRICKS_TOKEN")

print("HOST :", host)
print("TOKEN:", "Sí ✅" if token else "No ❌")

assert host and token, "🔥 ERROR: Faltan credenciales Databricks (HOST/TOKEN)."

# --- 5) Ping directo a API Databricks MLflow ---
url = host.rstrip("/") + "/api/2.0/mlflow/experiments/list"
r = requests.get(url, headers={"Authorization": f"Bearer {token}"}, timeout=20)
print("📡 /experiments/list ->", r.status_code)
if r.status_code != 200:
    print("❌ Detalle:", r.text[:400])
r.raise_for_status()

# --- 6) Configurar MLflow ---
mlflow.set_tracking_uri("databricks")
mlflow.set_registry_uri("databricks")
print("✅ MLflow tracking URI:", mlflow.get_tracking_uri())



📂 Repo root: C:\Users\cheve\nyc-taxi-predictionss
⚠️ No existe databricks_host.txt
HOST : None
TOKEN: Sí ✅


AssertionError: 🔥 ERROR: Faltan credenciales Databricks (HOST/TOKEN).